In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x220fae16ff0>)

In [3]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')


In [4]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력
print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력


훈련용 리뷰 개수 : 150000
테스트용 리뷰 개수 : 50000


In [5]:
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data = train_data.dropna(how='any') # Null 값이 존재하는 행 제거
print('훈련 데이터의 리뷰 수 :',len(train_data))


훈련 데이터의 리뷰 수 : 146182


In [6]:
test_data = test_data.dropna(how = 'any')
print('테스트 데이터의 리뷰 수 :',len(test_data))


테스트 데이터의 리뷰 수 : 49997


AutoTokenizer.from_pretrained('모델 이름')을 사용하면 모델 이름에 맞는 토크나이저를 자동으로 로드합니다. skt/kogpt2-base-v2의 토크나이저를 로드해봅시다.

In [7]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')


In [8]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['▁보는', '내', '내', '▁그대로', '▁들어', '맞', '는', '▁예측', '▁카', '리스', '마', '▁없는', '▁악', '역']


In [9]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))


[11867, 7071, 7071, 10554, 9359, 7498, 7162, 15305, 9488, 10191, 7487, 9712, 9868, 8031]


In [10]:
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))


'보는내내 그대로 들어맞는 예측 카리스마 없는 악역'

In [11]:
print(tokenizer.decode(3))


<pad>


패딩 토큰은 3번임을 확인했습니다. encode()를 할 때, 최대 길이를 지정하고 해당 길이까지 패딩하는 것도 가능합니다. max_length의 값으로 최대 길이를 지정해주고 pad_to_max_length의 값을 True로 하여 정수 인코딩을 하는 동시에 최대 길이까지 패딩해봅시다.

In [15]:
max_seq_len = 128

encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", padding='max_length', max_length=max_seq_len, truncation=True)
print(encoded_result)
print('길이 :', len(encoded_result))


[9034, 13555, 16447, 10584, 389, 9427, 10056, 7898, 8135, 10584, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
길이 : 128


전체 데이터에 대해서 전처리를 진행해봅시다. 전처리 과정에서 koGPT-2의 시작 토큰/종료 토큰을 리뷰의 앞 뒤에 부착합니다. 앞의 챗봇 구현 실습에서 확인한 바와 같이 해당 토큰은 </s>입니다. 그 후 정해진 최대 길이로 패딩을 진행합니다.

In [18]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token] # 시작토큰
        eos_token = [tokenizer.eos_token] # 종료토큰
        
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels


In [19]:
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 146182/146182 [00:11<00:00, 12315.35it/s]


In [20]:
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)


100%|██████████| 49997/49997 [00:04<00:00, 12417.40it/s]


In [21]:
# 최대 길이: 128
input_id = train_X[0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)


단어에 대한 정수 인코딩 : [    1  9050  9267  7700  9705 23971 12870  8262  7055  7098  8084 48213
     1     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : </s> 아 더빙.. 진짜 짜증나네요 목소리</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

## GPT의 출력 이해하기

koGPT-2를 이용해 모델을 구현하기 위해서는 koGPT-2의 출력을 이해할 필요가 있습니다. 우선, 한국어 GPT-2인 skt/kogpt2-base-v2를 로드해봅시다.

In [24]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
# todo 이렇게 하지 않으면 오류나네...
import tf_keras as keras

max_seq_len = 128

# input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
input_ids_layer = keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32) # 호환성 때문에 이렇게 해야 할 듯
outputs = model([input_ids_layer])

outputs에는 두 개의 출력이 존재하는데 인덱스 0을 확인해봅시다.

In [32]:
# 문장 길이만큼의 출력
print(outputs[0])


KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tfgpt2_model_1/transformer/Reshape_2:0', description="created by layer 'tfgpt2_model_1'")


outputs[0]은 (배치 크기, 128, 768)의 크기를 가지는 텐서입니다. 이는 768차원의 벡터가 128개가 있다는 의미로 문장 길이 개수만큼의 출력을 얻었음을 의미합니다. 텍스트 분류 문제를 풀 경우에는 koGPT-2의 마지막 예측에 해당하는 벡터를 사용해야 합니다.

In [33]:
print(outputs[0][:, -1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem_1/strided_slice:0', description="created by layer 'tf.__operators__.getitem_1'")


## GPT를 이용한 텍스트 분류 모델 만들기
1. 지금까지 정리하자면
2. 토큰으로 정수인코딩을 했고
3. 모델을 불러와서 샘플로 찍어 봤고(여기는 그냥 한번 해본것)
4. 아래 부터 파인튜닝하는 것(정수인코딩한 데이터로)

서브클래싱 구현 방식으로 구현한 텍스트 분류 모델은 다음과 같습니다. GPT의 출력 중 outputs[0][:, -1]. 즉, 마지막 출력 벡터를 시그모이드 함수가 활성화 함수로 설정된 출력층으로 연결합니다.

In [35]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs): # fit할 때 호출되는 함수
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction


In [36]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
# 시간이 많이 걸림
model.fit(train_X, train_y, epochs=2, batch_size=32, validation_split=0.2)

## 예측해보기

In [ ]:
def sentiment_predict(new_sentence):

  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
  input_id = np.array([input_id])
  
  # 이렇게 예측할 수 있는 함수를 제공하면 오죽 좋나. 그런데 대부분 제공한다. 다만 인풋 형식을 맞추어야 한다.
  score = model.predict(input_id)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))


In [ ]:
sentiment_predict("보던거라 계속보고있는데 전개도 느리고 주인공인 은희는 한두컷 나오면서 소극적인모습에")